In [ ]:
from flask import Flask, request, send_file, render_template
import openpyxl
import os

app = Flask(__name__)
UPLOAD_FOLDER = "uploads"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

def add_quotes_to_excel(input_path, output_path):
    workbook = openpyxl.load_workbook(input_path)
    for sheet in workbook.sheetnames:
        worksheet = workbook[sheet]
        for row in worksheet.iter_rows(min_row=2):  # Skip header
            for cell in row:
                if cell.value is not None:
                    cell.value = f'"{cell.value}"'
    workbook.save(output_path)

@app.route("/", methods=["GET", "POST"])
def upload_file():
    if request.method == "POST":
        file = request.files["file"]
        if file:
            input_path = os.path.join(UPLOAD_FOLDER, file.filename)
            output_path = os.path.join(UPLOAD_FOLDER, "output_" + file.filename)
            file.save(input_path)
            add_quotes_to_excel(input_path, output_path)
            return send_file(output_path, as_attachment=True)
    return render_template("index.html")

if __name__ == "__main__":
    app.run(debug=True)